# Notebook probando predecir la probabilidad de éxito de cobro

## Importar librerías

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil import parser

Cargar los datos

In [4]:
# Cargar los datos
df = pd.read_csv('../data/cobros_final.csv')

C:\Users\angel\AppData\Local\Temp\ipykernel_6596\1335326567.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/cobros_final.csv')


Contar nulos

In [5]:
# Count nulls
nulls = df.isnull().sum()
nulls

idListaCobro                0
idCredito                   0
consecutivoCobro            0
idBanco                     0
montoExigible               0
montoCobrar                 0
montoCobrado                0
fechaCobroBanco       9679374
idRespuestaBanco       149561
fechaCreacionLista          0
fechaEnvioCobro             0
idEmisora                   0
dtype: int64

# Generar una razón de cuantas veces no ha pagado un credito

In [6]:
# Group by idCredito and sum the times montoCobrado is different than 0
countnopagar = df[df['montoCobrado'] == 0].groupby('idCredito').size().reset_index(name='non_zero_count')
countnopagar


,idCredito,non_zero_count
0,6,2
1,560,1
2,773,2
3,1154,2
4,1315,2
...,...,...
90186,755612,9
90187,755650,6
90188,755676,1
90189,755859,16


In [7]:
counttotal = df.groupby('idCredito')["idBanco"].count()
counttotal

idCredito
6          2
560        1
773        2
1154       2
1315       2
          ..
755683     1
755695     1
755726     1
755859    16
756595     5
Name: idBanco, Length: 97230, dtype: int64

In [8]:
# Select the index that appear on countnopagar that are not on counttotal
siemprepago = counttotal[~counttotal.index.isin(countnopagar['idCredito'])]
siemprepago

idCredito
20873     1
46038     2
46590     1
54690     1
57667     4
         ..
755668    1
755672    1
755683    1
755695    1
755726    1
Name: idBanco, Length: 7039, dtype: int64

In [9]:
# Remove the rows on countottal that are on siemprepago
counttotal = counttotal[~counttotal.index.isin(siemprepago.index)].reset_index()
counttotal

,idCredito,idBanco
0,6,2
1,560,1
2,773,2
3,1154,2
4,1315,2
...,...,...
90186,755612,9
90187,755650,6
90188,755676,1
90189,755859,16


In [10]:
# Divide counttotal['idBanco'] by countnopagar['idBanco']
counttotal["razon"] =  1 - countnopagar['non_zero_count'] / counttotal['idBanco']
counttotal

,idCredito,idBanco,razon
0,6,2,0.0
1,560,1,0.0
2,773,2,0.0
3,1154,2,0.0
4,1315,2,0.0
...,...,...,...
90186,755612,9,0.0
90187,755650,6,0.0
90188,755676,1,0.0
90189,755859,16,0.0


In [11]:
# Add to df the column 'razon' with the values of counttotal['razon'] where df['idCredito'] == counttotal['idCredito']
df = df.merge(counttotal[['idCredito', 'razon']], on='idCredito', how='left')
# Fill the nulls on 'razon' with 1
df

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,fechaCreacionLista,fechaEnvioCobro,idEmisora,razon
0,100613,83021,27169966,BANAMEX,572.24,572.24,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.014320
1,100613,87227,27169937,BANAMEX,517.74,517.74,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.000000
2,100613,103785,27169946,BANAMEX,681.23,681.23,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.000000
3,100613,110463,27169969,BANAMEX,572.24,572.24,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.006897
4,100613,113755,27169948,BANAMEX,340.62,340.62,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.014134
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10057435,162271,741795,44299050,BANAMEX,980.32,980.32,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000
10057436,162271,741795,44299051,BANAMEX,1633.86,1633.86,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000
10057437,162271,742470,44299052,BANAMEX,810.44,810.44,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000
10057438,162271,742470,44299053,BANAMEX,1350.74,1350.74,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000


In [12]:
# Replace the nulls on 'razon' with 1
df['razon'] = df['razon'].fillna(1)

# Agregar variable objetivo

In [13]:
# Add a column to df 'pay' where 1 if montoCobrado > 0 else 0
df['pay'] = np.where(df['montoCobrado'] > 0, 1, 0)
df

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,fechaCreacionLista,fechaEnvioCobro,idEmisora,razon,pay
0,100613,83021,27169966,BANAMEX,572.24,572.24,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.014320,0
1,100613,87227,27169937,BANAMEX,517.74,517.74,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.000000,0
2,100613,103785,27169946,BANAMEX,681.23,681.23,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.000000,0
3,100613,110463,27169969,BANAMEX,572.24,572.24,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.006897,0
4,100613,113755,27169948,BANAMEX,340.62,340.62,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.014134,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10057435,162271,741795,44299050,BANAMEX,980.32,980.32,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000,0
10057436,162271,741795,44299051,BANAMEX,1633.86,1633.86,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000,0
10057437,162271,742470,44299052,BANAMEX,810.44,810.44,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000,0
10057438,162271,742470,44299053,BANAMEX,1350.74,1350.74,0.0,NaN,4.0,22/05/2025 12:10PM,22/05/2025,BANAMEX TRADICIONAL REINTENTO,0.000000,0


In [14]:
df.value_counts('pay')

pay
0    9679374
1     378066
Name: count, dtype: int64

In [15]:
df.head()

,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco,fechaCreacionLista,fechaEnvioCobro,idEmisora,razon,pay
0,100613,83021,27169966,BANAMEX,572.24,572.24,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.014320,0
1,100613,87227,27169937,BANAMEX,517.74,517.74,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.000000,0
2,100613,103785,27169946,BANAMEX,681.23,681.23,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.000000,0
3,100613,110463,27169969,BANAMEX,572.24,572.24,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.006897,0
4,100613,113755,27169948,BANAMEX,340.62,340.62,0.0,NaN,Cuenta Insuficiencia Fondos,30/09/2022 7:33AM,30/09/2022,Banamex Excepciones Cuenta ...,0.014134,0


Solamente quedarse con las columnas para entrenar

In [16]:
# Generate a new dataframe with the columns 'idCredito', 'razon', 'pay' and 'fechaCobro'
df2 = df[['idBanco', 'montoExigible', 'fechaEnvioCobro','idEmisora','pay']].copy()
df2 = pd.get_dummies(df2, columns=['idBanco', 'idEmisora'], drop_first=True)
df2.head()



,montoExigible,fechaEnvioCobro,pay,idBanco_AZTECA,idBanco_BABIEN,idBanco_BANAMEX,idBanco_BANCO DEL BAJIO,idBanco_BANCO FAMSA,idBanco_BANCOPPEL,idBanco_BANORTE,...,idEmisora_Banamex Reintentos,idEmisora_Banamex Tradicional Tarjeta Periodo Actual,idEmisora_Bancomer Parcial Cuenta Excepciones,idEmisora_Bancomer Reintentos,idEmisora_Bancomer Tradicional,idEmisora_Banorte Excepciones Cuenta,idEmisora_SANTANDER CLABE INTERBANCARIO,idEmisora_SANTANDER CLABE TRADICIONAL,idEmisora_SANTANDER CUENTA,idEmisora_Santander Excepciones Cuenta
0,572.24,30/09/2022,0,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,517.74,30/09/2022,0,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,681.23,30/09/2022,0,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,572.24,30/09/2022,0,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,340.62,30/09/2022,0,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [17]:

# Transform fechaEnvioCobro into month and day
df2['fechaEnvioCobro'] = pd.to_datetime(df2['fechaEnvioCobro'], format='%d/%m/%Y')
df2['month'] = df2['fechaEnvioCobro'].dt.month
df2['day'] = df2['fechaEnvioCobro'].dt.day
# Drop the column 'fechaEnvioCobro'
df2.drop(columns=['fechaEnvioCobro'], inplace=True)

## Entrenar el modelo

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

# Features and target
X = df2.drop(columns=['pay'])
y = df2['pay']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Create and train the model
model = GaussianNB()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.99      0.89      0.94   2903812
           1       0.19      0.68      0.30    113420

    accuracy                           0.88   3017232
   macro avg       0.59      0.79      0.62   3017232
weighted avg       0.96      0.88      0.91   3017232



In [25]:
X_test.iloc[0]

montoExigible                                                   163.15
idBanco_AZTECA                                                   False
idBanco_BABIEN                                                   False
idBanco_BANAMEX                                                   True
idBanco_BANCO DEL BAJIO                                          False
idBanco_BANCO FAMSA                                              False
idBanco_BANCOPPEL                                                False
idBanco_BANORTE                                                  False
idBanco_BANREGIO                                                 False
idBanco_BBVA MEXICO                                              False
idBanco_BMULTIVA                                                 False
idBanco_HSBC                                                     False
idBanco_INBURSA                                                  False
idBanco_SANTANDER                                                False
idBanc

In [29]:
y_test

7948533    0
8840712    0
9778407    0
4622602    0
8664911    0
          ..
6901284    0
2903200    0
8821755    0
4619216    0
6247279    0
Name: pay, Length: 3017232, dtype: int64

In [26]:
# Predict the first row
model.predict(X_test.iloc[0].values.reshape(1, -1))

d:\databallers\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([0])

In [19]:
X_test.loc[9900384]

montoExigible                                                   163.27
idBanco_AZTECA                                                   False
idBanco_BABIEN                                                   False
idBanco_BANAMEX                                                  False
idBanco_BANCO DEL BAJIO                                          False
idBanco_BANCO FAMSA                                              False
idBanco_BANCOPPEL                                                False
idBanco_BANORTE                                                  False
idBanco_BANREGIO                                                 False
idBanco_BBVA MEXICO                                              False
idBanco_BMULTIVA                                                 False
idBanco_HSBC                                                     False
idBanco_INBURSA                                                  False
idBanco_SANTANDER                                                 True
idBanc

In [21]:
model.predict_proba(X_test.loc[9900384].values.reshape(1, -1))

d:\databallers\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[1.1261155e-15, 1.0000000e+00]])

In [22]:
y_test.loc[9900384]

np.int64(1)

In [38]:
l=model.predict_proba(X_test)

In [39]:
l

array([[9.99999109e-01, 8.91267655e-07],
       [9.99996759e-01, 3.24086332e-06],
       [9.99999859e-01, 1.41255328e-07],
       ...,
       [9.99999278e-01, 7.21571271e-07],
       [9.99864890e-01, 1.35109999e-04],
       [1.00000000e+00, 1.74336223e-10]], shape=(3017232, 2))

In [33]:
model.predict_proba(X_test.loc[9900384].values.reshape(1, -1))

d:\databallers\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([[1.1261155e-15, 1.0000000e+00]])

In [161]:
# Save the df2 df
df2.to_csv('../data/dftransformado.csv', index=False)

In [162]:
df3=   df2.head(1000)
df3

,montoExigible,pay,idBanco_AZTECA,idBanco_BABIEN,idBanco_BANAMEX,idBanco_BANCO DEL BAJIO,idBanco_BANCO FAMSA,idBanco_BANCOPPEL,idBanco_BANORTE,idBanco_BANREGIO,...,idEmisora_Bancomer Parcial Cuenta Excepciones,idEmisora_Bancomer Reintentos,idEmisora_Bancomer Tradicional,idEmisora_Banorte Excepciones Cuenta,idEmisora_SANTANDER CLABE INTERBANCARIO,idEmisora_SANTANDER CLABE TRADICIONAL,idEmisora_SANTANDER CUENTA,idEmisora_Santander Excepciones Cuenta,month,day
0,572.24,0,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,9,30
1,517.74,0,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,9,30
2,681.23,0,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,9,30
3,572.24,0,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,9,30
4,340.62,0,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,9,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1382.25,0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,3,28
996,1103.59,0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,3,28
997,1777.82,0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,3,28
998,514.61,0,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,3,28


In [163]:
df3.to_csv('../data/dftransformado_1000.csv', index=False)

In [159]:
# Save the model
import joblib
joblib.dump(model, '../models/naive_bayes_model.pkl')

['../models/naive_bayes_model.pkl']